In [2]:
import os,sys
import math

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mlt
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation
import seaborn as sns
from sklearn.model_selection import train_test_split

sys.path.append("../src")  # adjust path as needed
from read_Parkinsonpredict import ReadData, LoadData
from ml_models import MlModels

### Load the Merged gene expression and MRI data

In [3]:
path = '/Users/pushpita/Documents/Erdos_bootcamp/our_project/Data/fomatted_data/'
loaddata = LoadData(input_path=path)
X_train, Y_train = loaddata.merged_data()

Is PATNO sorted?: True
Do we have all unique patients?: Yes
The train and test data is split according to 80.0 - 20.0%


### Loading the Logistic Regression model

##### Splitting the trainning data further into train and validation set for regular logistic regression

In [4]:
X_train_reg, X_cv, Y_train_reg, Y_cv = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True, random_state=64)
data = {}
data['X_train'] = X_train_reg
data['Y_train'] = Y_train_reg

In [5]:
my_model = MlModels(model_name = 'logistic', data=data, params = None, pca_flag=True, pca_components = 10)
mymodel_logistic = my_model.build_model()

/Users/pushpita/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [6]:
# Apply the same preprocessing (scaling + PCA) if used during training
if my_model.pca_flag:
    X_cv_scaled = my_model.scaler.transform(X_cv)
    X_cv_pca = my_model.pca.transform(X_cv_scaled)
    predictions = my_model.model.predict(X_cv_pca)
else:
    predictions = my_model.model.predict(X_cv)

# Now evaluate
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Validation Accuracy:", accuracy_score(Y_cv.values.ravel(), predictions))
print("Classification Report:\n", classification_report(Y_cv.values.ravel(), predictions))
print("Confusion Matrix:\n", confusion_matrix(Y_cv.values.ravel(), predictions))

Validation Accuracy: 0.5
Classification Report:
               precision    recall  f1-score   support

         0.0       0.53      0.44      0.48        18
         1.0       0.00      0.00      0.00         3
         2.0       0.51      0.83      0.63        23
         3.0       0.00      0.00      0.00         2
         4.0       0.00      0.00      0.00         6
         5.0       0.00      0.00      0.00         2

    accuracy                           0.50        54
   macro avg       0.17      0.21      0.19        54
weighted avg       0.40      0.50      0.43        54

Confusion Matrix:
 [[ 8  0  9  0  1  0]
 [ 1  0  2  0  0  0]
 [ 3  0 19  1  0  0]
 [ 0  0  2  0  0  0]
 [ 3  0  3  0  0  0]
 [ 0  0  2  0  0  0]]


/Users/pushpita/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/pushpita/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/pushpita/anaconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

##### Regular Logistic regression is a very bad model for this since the number of data points in the NHY score classes is heavily skewed